In [ ]:
!pip install dash_bootstrap_components

In [ ]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, Input, Output, callback

from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc
COMPONENT_STYLE = "/assets/my_component.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__,requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [ ]:
app.layout = html.Div(['Hello World'])

#app.run_server(host="0.0.0.0", port=5051)

In [ ]:
import pandas as pd

In [ ]:
from sqlalchemy import create_engine
#import psycopg2
engine = create_engine('mysql://pythonuser:pythonuser@db:3306/RubyDB')


In [ ]:
sqlquery = '''SELECT id, name FROM RubyDB.experiments;'''
experiments_df = pd.read_sql_query(sqlquery, engine)
experiments_df.head(5)

# Color Code

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

# colors
palette = cycle(px.colors.qualitative.Bold)
#palette = cycle(['black', 'grey', 'red', 'blue'])

# SQL

In [ ]:
sqlquery = '''SELECT id,data_label,data_values,data_comment,
data_reference FROM RubyDB.limits;'''

dv_df = pd.read_sql_query(sqlquery, engine)
dv_df

In [ ]:


lol = []
for index, row in dv_df.iterrows():
    #print(row['id'], row['data_values'])

    data_string = row[['data_values']].iloc[0]
    data_string = data_string.replace("{[", "")
    data_string = data_string.replace("]}", "")
    #print(data_string)
    data_series = data_string.split("]")
    #print(len(data_series))
    for l in range(0,len(data_series)):
        next_colour = next(palette)
        single_set = data_series[l]
        set_list = single_set.split(";")
        for i in set_list:
            z = i.split(" ");
            new_x = z[0].replace(",[", "")
            try:
                appendthis = [row['id'],l,new_x,z[1],next_colour]
            except:
                appendthis = [row['id'],l,0,0]
            lol.append(appendthis)
    #lol
df_experiment = pd.DataFrame(data=lol,columns=['experiment','series','raw_x','raw_y','series_color'])

df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(dtype = float, errors = 'ignore')
df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(dtype = float, errors = 'ignore')
   

In [ ]:
df_experiment[df_experiment['series']==7]['experiment'].unique()

In [ ]:
#df_experiment[df_experiment['series']==3]      

df_plot = df_experiment[df_experiment['experiment']==262].copy() 
#df_plot
fig = px.scatter(df_plot, x='x', y='y', color='series')

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig.show()
#new_df = dv_df.data_values.str.split(pat=']', n=- 1, expand=False)
#new_df.iloc[0]

In [ ]:
series_count = df_plot['series'].max()

df_plot

In [ ]:
dff = df_plot

fig1 = go.Figure()


for i in range(0,series_count):
    trace_data_df = df_plot[df_plot['series'] == i]
    
    fig1.add_trace(
    go.Scatter(
        x=trace_data_df['x'],
        y=trace_data_df['y'],
        name=trace_data_df['series'].iloc[0].astype(str),
        mode="lines",
        line=go.scatter.Line(color=trace_data_df['series_color'].iloc[0]),
        showlegend=True))
    
fig1.update_xaxes(type="log")
fig1.update_yaxes(type="log")

fig1.update_layout(
#xaxis_title=filtered_df['x_units'].iloc[0],
#yaxis_title=filtered_df['y_units'].iloc[0],
title="Multi Series",
legend_title="Legend Title",
font=dict(
    family="Courier New, monospace",
    size=18,
    color="RebeccaPurple"
)
)
    
fig1.show()

In [ ]:
##sqlquery = '''SELECT id, name FROM RubyDB.experiments;'''

sqlquery = '''SELECT
id, spin_dependency, result_type, measurement_type, nomhash, x_units, y_units, x_rescale,
y_rescale, default_color, default_style,
data_values, data_label, file_name, data_comment,
data_reference, created_at, updated_at, creator_id, experiment, rating, date_of_announcement,
public, official, date_official, greatest_hit, date_of_run_start, date_of_run_end, `year`
FROM RubyDB.limits;'''

limits_df = pd.read_sql_query(sqlquery, engine)
limits_df

# Drop Downs

In [ ]:
sqlquery = '''SELECT id, name FROM RubyDB.experiments;'''
experiments_df = pd.read_sql_query(sqlquery, engine)
experiments_df

#https://dash-bootstrap-components.opensource.faculty.ai/docs/components/form/

#x range (WIMP mass) 	lower bound: 	to 	upper bound: 	GeV/c^2
#y range (cross-section) 	smallest c-s: 10^ 	to 	largest c-s: 10^ 	
#Table
#Result	Reference	Plot appearance

ResultTypeDropdown = dcc.Dropdown(
        id='resulttype-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Experiment', 'value': 'Experiment'},
            {'label': 'Projection', 'value': 'Projection'},
            {'label': 'Theory', 'value': 'Theory'},
            {'label': 'Other', 'value': 'Other'},
            {'label': 'Personal', 'value': 'Personal'}            
        ],
        value='All'
    )

#dbc.DropdownMenu(
#    label="Menu",
#    children=[
#        dbc.DropdownMenuItem("Item 1"),
#        dbc.DropdownMenuItem("Item 2"),
#        dbc.DropdownMenuItem("Item 3"),
#    ],
#)

SpinDropdown = dcc.Dropdown(
        id='spin-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'spin-dependent', 'value': 'spin-dependent'},
            {'label': 'spin-indpendent', 'value': 'spin-indpendent'}    
        ],
        value='All'
    )


ExperimentDropdown = dcc.Dropdown(
        id='experiment-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'CDEX-10', 'value': ''},
            {'label': 'CDMS I (SUF)', 'value': ''},
            {'label': 'CDMS II (Soudan)', 'value': ''},
            {'label': 'CDMS II (Soudan)', 'value': ''},
            {'label': 'COSME', 'value': ''},
            {'label': 'COUPP', 'value': ''},
            {'label': 'COURICIONO', 'value': ''},
            {'label': 'CoGeNT', 'value': ''},
            {'label': 'Cuore', 'value': ''}
        ],
        value='All'
    )

YearDropdown = dcc.Dropdown(
        id='year-dropdown',
        options=[
            {'label': '2000', 'value': '2000'},
            {'label': '2001', 'value': '2001'},
            {'label': '2002', 'value': '2002'},
            {'label': '2003', 'value': '2003'},
            {'label': '2004', 'value': '2004'},
            {'label': '2005', 'value': '2005'},
            {'label': '2006', 'value': '2006'},
            {'label': '2007', 'value': '2007'},
            {'label': '2008', 'value': '2008'},
            {'label': '2009', 'value': '2009'},
            {'label': '2010', 'value': '2010'},
            {'label': '2011', 'value': '2011'},
            {'label': '2012', 'value': '2012'},
            {'label': '2013', 'value': '2013'},
            {'label': '2014', 'value': '2014'},
            {'label': '2015', 'value': '2015'},
            {'label': '2016', 'value': '2016'},
            {'label': '2017', 'value': '2017'},
            {'label': '2018', 'value': '2018'},
            {'label': '2019', 'value': '2019'},
            {'label': '2020', 'value': '2020'}
        ],
        value='All'
    )

GreatestHitsDropdown = dcc.Dropdown(
        id='greatesthits-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Greatest Hits', 'value': 'Greatest Hits'}
        ],
        value='All'
    )

#List of Research Results

#Result	Reference

In [ ]:
experiments_sql = '''select distinct experiment as label, experiment as value 
FROM RubyDB.limits;'''


result_types_sql = '''SELECT distinct
case
when result_type = "Th" then "Theory"
when result_type = "Proj" then "Project"
when result_type = "Exp" then "Experiment"
else result_type end label,
result_type as value
FROM RubyDB.limits;'''

spin_dependency_sql = '''SELECT distinct
case
when spin_dependency = 'SD' then 'spin-dependent'
when spin_dependency = 'SI' then 'spin-indpendent'
else spin_dependency end label,
spin_dependency as value
FROM RubyDB.limits;'''

greatest_hit_sql = '''select distinct
case
when greatest_hit = 0 then 'No'
when greatest_hit = 1 then 'Yes'
else greatest_hit end label,
 greatest_hit value
FROM RubyDB.limits;'''

In [ ]:
def get_dropdown(id_in, sql_query_in, default_in):
    dropdown_df = pd.read_sql_query(sql_query_in, engine)
    dropdown_df = dropdown_df.fillna('XXXX')
    dropdown_df_copy = dropdown_df.copy()
    dropdown_dict = dropdown_df_copy.to_dict('records')
    dropdown = dcc.Dropdown(
        id=id_in,
        options=dropdown_dict,
        value=default_in
    )
    return dropdown

In [ ]:
experiment_dropdown = get_dropdown(id_in='experiment_dropdown',sql_query_in=experiments_sql,default_in='All')
resulttype_dropdown = get_dropdown('result_type_dropdown',result_types_sql,'Th')
spin_dropdown = get_dropdown('spin_dependency_dropdown',spin_dependency_sql,'SI')
greatesthits_dropdown = get_dropdown('greatest_hit_dropdown',greatest_hit_sql,0)

In [ ]:
#greatesthits_dropdown
#experiment_dropdown
resulttype_dropdown

In [ ]:
#experiment_dropdown = get_dropdown(id_in='exeriment_dropdown',sql_query_in=experiments_sql,default_in='Theory')
#resulttype_dropdown = get_dropdown('result_type_dropdown',result_types_sql,'All')
#spin_dropdown = get_dropdown('spin_dependency_dropdown',spin_dependency_sql,'Theory')
#greatesthits_dropdown = get_dropdown('greatest_hit_dropdown',greatest_hit_sql,0)

import dash_bootstrap_components as dbc

form = dbc.Container(
    children=[dbc.Row(
    [
        dbc.Col(
            [
                dbc.Label("Result Type"),
                resulttype_dropdown,
            ],
            width=2,
        ),
        dbc.Col(
            [
                dbc.Label("Spin"),
                spin_dropdown,
            ],
            width=2,
        ),
        dbc.Col(
            [
                dbc.Label("Experiment"),
                experiment_dropdown,
            ],
            width=2,
        ),
        dbc.Col(
            [
                dbc.Label("Greatest Hits"),
                greatesthits_dropdown,
            ],
            width=2,
        ),
    ],
    className="g-3",
)])

In [ ]:
#app.layout = form
#app.run_server(host="0.0.0.0", port=5051)
#app.run_server(debug=True,port=5051)

In [ ]:
#app.run_server(mode='inline',host='0.0.0.0', port='5051')

#app.run_server(mode='inline', host="0.0.0.0", port=5051)

In [ ]:
dv = limits_df[limits_df['id']==100][['data_values']]
oneid = limits_df[limits_df['id']==100]

In [ ]:
allid = limits_df.copy()
#limits_df.columns
allid = allid[[
'id',
'data_values',         
'data_comment',
'data_label',
'data_reference',
'date_of_announcement',
'date_of_run_end',
'date_of_run_start',
'default_color',
'default_style',
'experiment',
'public',
'result_type',
'spin_dependency',
'x_rescale',
'x_units',
'y_rescale',
'y_units',
'year']]
allid.dtypes

In [ ]:
selected_id_set = [100]
    
#value_list = ["apple"]
boolean_series = allid.id.isin(selected_id_set)

filtered_df = allid[boolean_series]

filtered_df

In [ ]:
#plot_df.to_dict('records')
df = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])

row = next(df.iterrows())[1]

type(row)

print(row)


In [ ]:
def datastring2dataframe(data_values_in):
    #try:
    #data_list = data_values_in['data_values'].tolist()
    if isinstance(data_values_in, pd.DataFrame):
        row_data = next(data_values_in.iterrows())[1]
    else:
        row_data = data_values_in
    data_string = row_data[['data_values']].iloc[0]
    data_string = data_string.replace("{[", "")
    data_string = data_string.replace("]}", "")
    x = data_string.split(";")
    lol = []
    for i in x:
        z = i.split(" ");
        appendthis = [z[0],z[1]]
        lol.append(appendthis)
    #lol
    df_experiment = pd.DataFrame(data=lol,columns=['raw_x','raw_y'])

    df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(dtype = float, errors = 'ignore')
    df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(dtype = float, errors = 'ignore')
    #except:
    #    data_null = [[0,0]]
    #    df_experiment = pd.DataFrame(data=data_null,columns=['raw_x','raw_y'])#

    #    df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(float)
    #    df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(float)
    #df_experiment.dtypes
    return df_experiment

In [ ]:
plot_df = allid
#plot_df = oneid

data_table = dash_table.DataTable(
        id='datatable-row-ids',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in plot_df.columns
            # omit the id column
            if i not in ['data_comment','spin_dependency', 'measurement_type', 'nomhash',
       'x_units', 'y_units', 'x_rescale', 'y_rescale', 'default_color',
       'default_style', 'data_values', 'file_name',
        'created_at', 'updated_at',
       'creator_id', 'rating', 'date_of_announcement', 'public',
       'official', 'date_official', 'greatest_hit', 'date_of_run_start',
       'date_of_run_end']
        ],
        data=plot_df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode='multi',
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
        page_action='native',
        page_current= 0,
        page_size= 10,
        derived_virtual_selected_row_ids=[],
        selected_row_ids=[],
        style_cell = {'font_family': 'Arial','font_size': '12px',},
        style_data={'whiteSpace': 'normal','height': 'auto',},
        style_cell_conditional=[
            {'if': {'column_id': 'id'},
             'width': '2%'},
            {'if': {'column_id': 'data_label'},
             'width': '50%'},
            {'if': {'column_id': 'data_reference'},
             'width': '20%'},
            {'if': {'column_id': 'experiment'},
             'width': '10%'},
            {'if': {'column_id': 'year'},
             'width': '5%'},
        ],
    )

app.layout = html.Div([
    data_table,
    html.Div(id='datatable-row-ids-container')
])



@app.callback(
    Output(component_id='datatable-row-ids-container', component_property='children'),
    Input('datatable-row-ids', 'derived_virtual_row_ids'),
    Input('datatable-row-ids', 'selected_row_ids'),
    Input('datatable-row-ids', 'active_cell'))
def update_graphs(row_ids, selected_row_ids, active_cell):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    
    #derived_virtual_data=data_df.to_dict('records')
    
    #selected_id_set = set(selected_row_ids or [])
    
    #value_list = ["apple"]

    boolean_series = plot_df.id.isin(selected_row_ids)

    filtered_df = plot_df[boolean_series].copy()
    
    #plot_df[plot_df['id']==100]

    plot_data = datastring2dataframe(filtered_df.iloc[0]).copy()
    
    if row_ids is None:
        dff = plot_data
        # pandas Series works enough like a list for this to be OK
        #row_ids = filtered_df['id']
        row_ids = [0]
    else:
        #dff = data_df.loc[row_ids]
        dff = plot_data
    
    
    active_row_id = active_cell['row'] if active_cell else None
    
    #dc = filtered_df['data_comment'][0]
    
    #print(dc)
    
    fig = go.Figure(
    data=[go.Scatter(x=dff['x'], y=dff['y'])],
    layout=go.Layout(
        #title=go.layout.Title(text="A Figure Specified By A Graph Object")
        ##title=go.layout.Title(text='y')
        title=go.layout.Title(text=filtered_df['data_label'].iloc[0])
        ##df_test['Btime'].iloc[0]
    )
)
    
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    
    fig.update_layout(
    xaxis_title=filtered_df['x_units'].iloc[0],
    yaxis_title=filtered_df['y_units'].iloc[0],
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
    
    
    graph = html.Div(children=[dcc.Graph(figure=fig)])
    
    
    '''graph = html.Div(children=[dcc.Graph(
            #id=column + '--row-ids',
            figure={
                'data': [
                    {
                        'x': dff['x'],
                        'y': dff['y'],
                        'type': 'scatter'#,
                        #'marker': {'color': colors},
                    }
                ],
                'layout': {
                    'xaxis': {'automargin': True},
                    'yaxis': {
                        'automargin': True,
                        'title': 'y',
                    },
                    'height': 250,
                    'margin': {'t': 10, 'l': 10, 'r': 10},
                },
            },
        )])'''
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        #for column in ['pop', 'lifeExp', 'gdpPercap'] if column in dff
        #for column in ['data_values'] if column in dff
    
    return graph


@app.callback(
    Output(component_id='datatable-row-ids-container', component_property='children'),
    Input('datatable-row-ids', 'derived_virtual_row_ids'),
    Input('datatable-row-ids', 'selected_row_ids'),
    Input('datatable-row-ids', 'active_cell'))
def update_graphs(row_ids, selected_row_ids, active_cell):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    
    #derived_virtual_data=data_df.to_dict('records')
    
    #selected_id_set = set(selected_row_ids or [])
    
    #value_list = ["apple"]

    boolean_series = plot_df.id.isin(selected_row_ids)

    filtered_df = plot_df[boolean_series].copy()
    
    #plot_df[plot_df['id']==100]

    plot_data = datastring2dataframe(filtered_df[['data_values']]).copy()
    
    '''if row_ids is None:
        dff = plot_data
        # pandas Series works enough like a list for this to be OK
        #row_ids = filtered_df['id']
        row_ids = [0]
    else:
        #dff = data_df.loc[row_ids]
        dff = plot_data.loc[row_ids]
    '''
    
    active_row_id = active_cell['row'] if active_cell else None
    
    #dc = filtered_df['data_comment'][0]
    
    fig = go.Figure(
    data=[go.Scatter(x=dff['x'], y=dff['y'])],
    layout=go.Layout(
        title=go.layout.Title(text="A Figure Specified By A Graph Object")
        ##title=go.layout.Title(text='y')
    )
)
    
    
    graph = html.Div(children=[dcc.Graph(figure=fig)])
    
    
    '''graph = html.Div(children=[dcc.Graph(
            #id=column + '--row-ids',
            figure={
                'data': [
                    {
                        'x': dff['x'],
                        'y': dff['y'],
                        'type': 'scatter'#,
                        #'marker': {'color': colors},
                    }
                ],
                'layout': {
                    'xaxis': {'automargin': True},
                    'yaxis': {
                        'automargin': True,
                        'title': 'y',
                    },
                    'height': 250,
                    'margin': {'t': 10, 'l': 10, 'r': 10},
                },
            },
        )])'''
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        #for column in ['pop', 'lifeExp', 'gdpPercap'] if column in dff
        #for column in ['data_values'] if column in dff
    
    return graph

In [ ]:
#app.run_server(debug=True,host='0.0.0.0', port=5051)

In [ ]:
#app.run_server(mode='jupyterlab', port = 5052, dev_tools_ui=True, #debug=True,
#              dev_tools_hot_reload =True, threaded=True)#

In [ ]:
app.run_server(host="0.0.0.0", port=5051)